## Data Analysis - Goal: to join supply and demand information about ethanol

In [47]:
import pandas as pd
import numpy as np

In [48]:
# To connect to the csv file that is downlaoded in the ANP web site
# https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/producao-de-biocombustiveis
# https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/vendas-de-derivados-de-petroleo-e-biocombustiveis
demand = pd.read_csv(
    'vendas-combustiveis-segmento-m3-2012-2025.csv', 
    sep=';',
    decimal=',',
    thousands='.'
    )
supply = pd.read_csv(
    'producao-etanol-anidro-hidratado-m3-2012-2025.csv',
    sep=';',
    decimal=',',
    thousands='.'
)

In [49]:
start_year = 2012
end_year = 2024

In [50]:
print(demand.head())
print(demand.tail())

    ANO  MÊS UNIDADE DA FEDERAÇÃO           PRODUTO          SEGMENTO  VENDAS
0  2012  JAN             RONDÔNIA  ETANOL HIDRATADO  POSTO REVENDEDOR  1560.5
1  2012  MAI     DISTRITO FEDERAL  ETANOL HIDRATADO  CONSUMIDOR FINAL    50.0
2  2012  JUN     DISTRITO FEDERAL  ETANOL HIDRATADO  CONSUMIDOR FINAL    25.0
3  2012  JUL     DISTRITO FEDERAL  ETANOL HIDRATADO  CONSUMIDOR FINAL    25.0
4  2012  AGO     DISTRITO FEDERAL  ETANOL HIDRATADO  CONSUMIDOR FINAL    20.0
        ANO  MÊS UNIDADE DA FEDERAÇÃO      PRODUTO          SEGMENTO  \
38146  2025  JAN   MATO GROSSO DO SUL  ÓLEO DIESEL  POSTO REVENDEDOR   
38147  2025  JAN    RIO GRANDE DO SUL  ÓLEO DIESEL  POSTO REVENDEDOR   
38148  2025  JAN       SANTA CATARINA  ÓLEO DIESEL  POSTO REVENDEDOR   
38149  2025  JAN            SÃO PAULO  ÓLEO DIESEL  POSTO REVENDEDOR   
38150  2025  JAN     DISTRITO FEDERAL  ÓLEO DIESEL               TRR   

           VENDAS  
38146   71057.868  
38147  206235.406  
38148  185694.560  
38149  532111.030  

In [51]:
print(supply.head())
print(supply.tail())

    ANO  MÊS GRANDE REGIÃO UNIDADE DA FEDERAÇÃO PRODUTO  PRODUÇÃO
0  2012  JAN  REGIÃO NORTE             RONDONIA  ANIDRO       0.0
1  2012  FEV  REGIÃO NORTE             RONDONIA  ANIDRO       0.0
2  2012  MAR  REGIÃO NORTE             RONDONIA  ANIDRO       0.0
3  2012  ABR  REGIÃO NORTE             RONDONIA  ANIDRO       0.0
4  2012  MAI  REGIÃO NORTE             RONDONIA  ANIDRO       0.0
       ANO  MÊS    GRANDE REGIÃO UNIDADE DA FEDERAÇÃO    PRODUTO   PRODUÇÃO
8350  2025  JAN     REGIÃO NORTE              RORAIMA  HIDRATADO      0.000
8351  2025  JAN       REGIÃO SUL       SANTA CATARINA  HIDRATADO      0.000
8352  2025  JAN   REGIÃO SUDESTE            SÃO PAULO  HIDRATADO   2277.512
8353  2025  JAN  REGIÃO NORDESTE              SERGIPE  HIDRATADO  10193.133
8354  2025  JAN     REGIÃO NORTE            TOCANTINS  HIDRATADO      0.000


In [52]:
demand_filter_product = demand[(demand['PRODUTO'] == 'GASOLINA C') | (demand['PRODUTO'] == 'ETANOL HIDRATADO')]

In [53]:
# Create a columns for the number month
month_numbers = {'JAN' : '01',
                 'FEV' : '02',
                 'MAR' : '03',
                 'ABR' : '04',
                 'MAI' : '05',
                 'JUN' : '06',
                 'JUL' : '07',
                 'AGO' : '08',
                 'SET' : '09',
                 'OUT' : '10',
                 'NOV' : '11',
                 'DEZ' : '12',
                 }
demand_filter_product = demand_filter_product.copy()
demand_filter_product['MONTH'] = demand_filter_product['MÊS'].map(month_numbers)
demand_filter_product.head()

,ANO,MÊS,UNIDADE DA FEDERAÇÃO,PRODUTO,SEGMENTO,VENDAS,MONTH
0,2012,JAN,RONDÔNIA,ETANOL HIDRATADO,POSTO REVENDEDOR,1560.5,01
1,2012,MAI,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,50.0,05
2,2012,JUN,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,25.0,06
3,2012,JUL,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,25.0,07
4,2012,AGO,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,20.0,08


In [54]:
demand_filter_product = demand_filter_product.copy()
demand_filter_product['Date'] = pd.to_datetime(demand_filter_product['ANO'].astype(str) + '-' + demand_filter_product['MONTH'].astype(str) + '-01')
demand_filter_product.head()

,ANO,MÊS,UNIDADE DA FEDERAÇÃO,PRODUTO,SEGMENTO,VENDAS,MONTH,Date
0,2012,JAN,RONDÔNIA,ETANOL HIDRATADO,POSTO REVENDEDOR,1560.5,01,2012-01-01
1,2012,MAI,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,50.0,05,2012-05-01
2,2012,JUN,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,25.0,06,2012-06-01
3,2012,JUL,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,25.0,07,2012-07-01
4,2012,AGO,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,20.0,08,2012-08-01


In [55]:
federative_unit = {
    'ACRE' : 'AC',
    'ALAGOAS' : 'AL',
    'AMAPÁ' : 'AP',
    'AMAZONAS' : 'AM',
    'BAHIA' : 'BA',
    'CEARÁ' : 'CE',
    'DISTRITO FEDERAL' : 'DF',
    'ESPÍRITO SANTO' : 'ES',
    'GOIÁS' : 'GO',
    'MARANHÃO' : 'MA',
    'MATO GROSSO' : 'MT',
    'MATO GROSSO DO SUL' : 'MS',
    'MINAS GERAIS' : 'MG',
    'PARANÁ' : 'PR',
    'PARAÍBA' : 'PB',
    'PARÁ' : 'PA',
    'PARA' : 'PA',
    'PERNAMBUCO' : 'PE',
    'PIAUÍ' : 'PI',
    'RIO DE JANEIRO' : 'RJ',
    'RIO GRANDE DO SUL' : 'RS',
    'RIO GRANDE DO NORTE' : 'RN',
    'RONDÔNIA' : 'RO',
    'RONDONIA' : 'RO',
    'RORAIMA' : 'RR',
    'SANTA CATARINA' : 'SC',
    'SERGIPE' : 'SE',
    'SÃO PAULO' : 'SP',
    'TOCANTINS' : 'TO'
}
demand_filter_product = demand_filter_product.copy()
demand_filter_product['UF'] = demand_filter_product['UNIDADE DA FEDERAÇÃO'].map(federative_unit)
demand_filter_product.head()

,ANO,MÊS,UNIDADE DA FEDERAÇÃO,PRODUTO,SEGMENTO,VENDAS,MONTH,Date,UF
0,2012,JAN,RONDÔNIA,ETANOL HIDRATADO,POSTO REVENDEDOR,1560.5,01,2012-01-01,RO
1,2012,MAI,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,50.0,05,2012-05-01,DF
2,2012,JUN,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,25.0,06,2012-06-01,DF
3,2012,JUL,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,25.0,07,2012-07-01,DF
4,2012,AGO,DISTRITO FEDERAL,ETANOL HIDRATADO,CONSUMIDOR FINAL,20.0,08,2012-08-01,DF


In [56]:
demand_filter_product_time = demand_filter_product.copy()
demand_filter_product_time = demand_filter_product_time[(demand_filter_product_time['ANO'] >= start_year) & (demand_filter_product_time['ANO'] <= end_year)]

In [57]:
ethanol_blend = {
    2014: 25.0,
    2015: 27.0,
    2016: 27.0,
    2017: 27.0,
    2018: 27.0,
    2019: 27.0,
    2020: 27.0,
    2021: 27.0,
    2022: 27.0,
    2023: 27.5,
    2024: 27.5
}

In [58]:
demand_filter_product_time

first_group_demand = demand_filter_product_time.groupby(['Date', 'UF', 'PRODUTO'])['VENDAS'].sum().reset_index()
first_group_demand

,Date,UF,PRODUTO,VENDAS
0,2012-01-01,AC,ETANOL HIDRATADO,453.000
1,2012-01-01,AC,GASOLINA C,8517.000
2,2012-01-01,AL,ETANOL HIDRATADO,3766.000
3,2012-01-01,AL,GASOLINA C,28949.500
4,2012-01-01,AM,ETANOL HIDRATADO,2913.000
...,...,...,...,...
8419,2024-12-01,SE,GASOLINA C,39897.000
8420,2024-12-01,SP,ETANOL HIDRATADO,901765.135
8421,2024-12-01,SP,GASOLINA C,785378.176
8422,2024-12-01,TO,ETANOL HIDRATADO,5654.800


In [59]:
second_group_demand = first_group_demand.loc[
    first_group_demand['PRODUTO'] == 'GASOLINA C'
].copy()
# Converter a coluna de data se necessário
second_group_demand['Date'] = pd.to_datetime(second_group_demand['Date'])

# Extrair o ano e mapear o blend de etanol
second_group_demand['YEAR'] = second_group_demand['Date'].dt.year
second_group_demand['ETANOL_BLEND'] = second_group_demand['YEAR'].map(ethanol_blend)

# Criar DataFrame final com as colunas necessárias
second_group_demand['VENDAS_ANIDRO'] = (second_group_demand['VENDAS'] * second_group_demand['ETANOL_BLEND'] / 100).round(2)
demand_anhydrous = second_group_demand[['Date', 'UF', 'PRODUTO', 'VENDAS_ANIDRO']].copy()
demand_anhydrous['PRODUTO'] = 'ANIDRO'
demand_anhydrous

,Date,UF,PRODUTO,VENDAS_ANIDRO
1,2012-01-01,AC,ANIDRO,NaN
3,2012-01-01,AL,ANIDRO,NaN
5,2012-01-01,AM,ANIDRO,NaN
7,2012-01-01,AP,ANIDRO,NaN
9,2012-01-01,BA,ANIDRO,NaN
...,...,...,...,...
8415,2024-12-01,RS,ANIDRO,99437.76
8417,2024-12-01,SC,ANIDRO,80070.43
8419,2024-12-01,SE,ANIDRO,10971.68
8421,2024-12-01,SP,ANIDRO,215979.00


In [60]:
supply = supply.copy() # Create a copy of the dataframe to avoid modifying the original data
supply = supply[(supply['ANO'] >= start_year) & (supply['ANO'] <= end_year)] # Filter the dataframe for the analysis period based on the start and end year inputs  
supply['UF'] = supply['UNIDADE DA FEDERAÇÃO'].map(federative_unit) # Map federal unit names to their corresponding abbreviations (UF)  
supply['MONTH'] = supply['MÊS'].map(month_numbers) # Convert month names to their corresponding numerical values  
supply['Date'] = pd.to_datetime(supply['ANO'].astype(str) + '-' + supply['MONTH'].astype(str) + '-01') # Create a datetime column using year and month (set day as 1 for consistency)  
first_group_supply = supply.groupby(['Date', 'UF', 'PRODUTO'])['PRODUÇÃO'].sum().reset_index() # Group by Date, UF, and PRODUTO, then sum the PRODUÇÃO values  
first_group_supply

,Date,UF,PRODUTO,PRODUÇÃO
0,2012-01-01,AC,ANIDRO,0.000
1,2012-01-01,AC,HIDRATADO,0.000
2,2012-01-01,AL,ANIDRO,64751.225
3,2012-01-01,AL,HIDRATADO,51371.502
4,2012-01-01,AM,ANIDRO,0.000
...,...,...,...,...
6785,2024-12-01,SE,HIDRATADO,10193.209
6786,2024-12-01,SP,ANIDRO,57260.399
6787,2024-12-01,SP,HIDRATADO,134640.894
6788,2024-12-01,TO,ANIDRO,126.785


In [61]:
demand_anhydrous = demand_anhydrous.rename(columns={ 'VENDAS_ANIDRO' : 'VENDAS'}).copy()
demand_SanD = pd.concat([first_group_demand, demand_anhydrous], ignore_index=True)
demand_SanD

,Date,UF,PRODUTO,VENDAS
0,2012-01-01,AC,ETANOL HIDRATADO,453.00
1,2012-01-01,AC,GASOLINA C,8517.00
2,2012-01-01,AL,ETANOL HIDRATADO,3766.00
3,2012-01-01,AL,GASOLINA C,28949.50
4,2012-01-01,AM,ETANOL HIDRATADO,2913.00
...,...,...,...,...
12631,2024-12-01,RS,ANIDRO,99437.76
12632,2024-12-01,SC,ANIDRO,80070.43
12633,2024-12-01,SE,ANIDRO,10971.68
12634,2024-12-01,SP,ANIDRO,215979.00


In [62]:
# demand_SandD_ethanol = demand_SanD[(demand_SanD['PRODUTO'] == 'ETANOL HIDRATADO') | (demand_SanD['PRODUTO'] == 'ANIDRO')]
# demand_SandD_ethanol = demand_SandD_ethanol.copy()
# # demand_SandD_ethanol
# demand_SandD_ethanol['KEY'] = demand_SandD_ethanol['Date'].astype(str) + '-' + demand_SandD_ethanol['UF'] + '-' + demand_SandD_ethanol['PRODUTO']
# demand_SandD_ethanol

# Filter for ethanol products only (hydrated and anhydrous)
demand_SandD_ethanol = demand_SanD[
    demand_SanD['PRODUTO'].isin(['ETANOL HIDRATADO', 'ANIDRO'])
].copy()  # Create independent copy to avoid warnings

# Create unique composite key: Date-UF-PRODUTO
demand_SandD_ethanol['KEY'] = (
    demand_SandD_ethanol['Date'].astype(str) + '-' + 
    demand_SandD_ethanol['UF'] + '-' + 
    demand_SandD_ethanol['PRODUTO']
)
demand_SandD_ethanol['PRODUTO'] = demand_SandD_ethanol['PRODUTO'].replace(
    {'ETANOL HIDRATADO': 'HIDRATADO'}
)
demand_SandD_ethanol

,Date,UF,PRODUTO,VENDAS,KEY
0,2012-01-01,AC,HIDRATADO,453.000,2012-01-01-AC-ETANOL HIDRATADO
2,2012-01-01,AL,HIDRATADO,3766.000,2012-01-01-AL-ETANOL HIDRATADO
4,2012-01-01,AM,HIDRATADO,2913.000,2012-01-01-AM-ETANOL HIDRATADO
6,2012-01-01,AP,HIDRATADO,302.000,2012-01-01-AP-ETANOL HIDRATADO
8,2012-01-01,BA,HIDRATADO,19579.835,2012-01-01-BA-ETANOL HIDRATADO
...,...,...,...,...,...
12631,2024-12-01,RS,ANIDRO,99437.760,2024-12-01-RS-ANIDRO
12632,2024-12-01,SC,ANIDRO,80070.430,2024-12-01-SC-ANIDRO
12633,2024-12-01,SE,ANIDRO,10971.680,2024-12-01-SE-ANIDRO
12634,2024-12-01,SP,ANIDRO,215979.000,2024-12-01-SP-ANIDRO


In [63]:
first_group_supply['KEY'] = (
    first_group_supply['Date'].astype(str) + '-' +
    first_group_supply['UF'] + '-' +
    first_group_supply['PRODUTO']
)
first_group_supply

,Date,UF,PRODUTO,PRODUÇÃO,KEY
0,2012-01-01,AC,ANIDRO,0.000,2012-01-01-AC-ANIDRO
1,2012-01-01,AC,HIDRATADO,0.000,2012-01-01-AC-HIDRATADO
2,2012-01-01,AL,ANIDRO,64751.225,2012-01-01-AL-ANIDRO
3,2012-01-01,AL,HIDRATADO,51371.502,2012-01-01-AL-HIDRATADO
4,2012-01-01,AM,ANIDRO,0.000,2012-01-01-AM-ANIDRO
...,...,...,...,...,...
6785,2024-12-01,SE,HIDRATADO,10193.209,2024-12-01-SE-HIDRATADO
6786,2024-12-01,SP,ANIDRO,57260.399,2024-12-01-SP-ANIDRO
6787,2024-12-01,SP,HIDRATADO,134640.894,2024-12-01-SP-HIDRATADO
6788,2024-12-01,TO,ANIDRO,126.785,2024-12-01-TO-ANIDRO


In [64]:
supply_and_demand_SandD_ethanol = demand_SandD_ethanol.merge(
    first_group_supply, 
    how='outer',
    on='KEY',
    suffixes=('_d', '_s')
    )
supply_and_demand_SandD_ethanol

,Date_d,UF_d,PRODUTO_d,VENDAS,KEY,Date_s,UF_s,PRODUTO_s,PRODUÇÃO
0,2012-01-01,AC,ANIDRO,NaN,2012-01-01-AC-ANIDRO,2012-01-01,AC,ANIDRO,0.000
1,2012-01-01,AC,HIDRATADO,453.000,2012-01-01-AC-ETANOL HIDRATADO,NaT,NaN,NaN,NaN
2,NaT,NaN,NaN,NaN,2012-01-01-AC-HIDRATADO,2012-01-01,AC,HIDRATADO,0.000
3,2012-01-01,AL,ANIDRO,NaN,2012-01-01-AL-ANIDRO,2012-01-01,AL,ANIDRO,64751.225
4,2012-01-01,AL,HIDRATADO,3766.000,2012-01-01-AL-ETANOL HIDRATADO,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
11791,2024-12-01,SP,HIDRATADO,901765.135,2024-12-01-SP-ETANOL HIDRATADO,NaT,NaN,NaN,NaN
11792,NaT,NaN,NaN,NaN,2024-12-01-SP-HIDRATADO,2024-12-01,SP,HIDRATADO,134640.894
11793,2024-12-01,TO,ANIDRO,10814.870,2024-12-01-TO-ANIDRO,2024-12-01,TO,ANIDRO,126.785
11794,2024-12-01,TO,HIDRATADO,5654.800,2024-12-01-TO-ETANOL HIDRATADO,NaT,NaN,NaN,NaN


In [65]:
# First fill UF_s and UF_d from whichever column exists
supply_and_demand_SandD_ethanol['UF_d'] = supply_and_demand_SandD_ethanol['UF_d'].fillna(supply_and_demand_SandD_ethanol['UF_s'])
supply_and_demand_SandD_ethanol['UF_s'] = supply_and_demand_SandD_ethanol['UF_s'].fillna(supply_and_demand_SandD_ethanol['UF_d'])
supply_and_demand_SandD_ethanol['PRODUTO_s'] = supply_and_demand_SandD_ethanol['PRODUTO_s'].fillna(supply_and_demand_SandD_ethanol['PRODUTO_d'])
supply_and_demand_SandD_ethanol['PRODUTO_d'] = supply_and_demand_SandD_ethanol['PRODUTO_d'].fillna(supply_and_demand_SandD_ethanol['PRODUTO_s'])
supply_and_demand_SandD_ethanol['Date_s'] = supply_and_demand_SandD_ethanol['Date_s'].fillna(supply_and_demand_SandD_ethanol['Date_d'])
supply_and_demand_SandD_ethanol['Date_d'] = supply_and_demand_SandD_ethanol['Date_d'].fillna(supply_and_demand_SandD_ethanol['Date_s'])

# Then fill all remaining numeric columns with 0
numeric_cols = ['VENDAS', 'PRODUÇÃO']
supply_and_demand_SandD_ethanol[numeric_cols] = supply_and_demand_SandD_ethanol[numeric_cols].fillna(0)

# For date columns, you might want to keep as NaT or fill with a default date
# supply_and_demand_SandD_ethanol['Date_d'] = supply_and_demand_SandD_ethanol['Date_d'].fillna(pd.NaT)
# supply_and_demand_SandD_ethanol['Date_s'] = supply_and_demand_SandD_ethanol['Date_s'].fillna(pd.NaT)
supply_and_demand_SandD_ethanol.head(10)

,Date_d,UF_d,PRODUTO_d,VENDAS,KEY,Date_s,UF_s,PRODUTO_s,PRODUÇÃO
0,2012-01-01,AC,ANIDRO,0.0,2012-01-01-AC-ANIDRO,2012-01-01,AC,ANIDRO,0.000
1,2012-01-01,AC,HIDRATADO,453.0,2012-01-01-AC-ETANOL HIDRATADO,2012-01-01,AC,HIDRATADO,0.000
2,2012-01-01,AC,HIDRATADO,0.0,2012-01-01-AC-HIDRATADO,2012-01-01,AC,HIDRATADO,0.000
3,2012-01-01,AL,ANIDRO,0.0,2012-01-01-AL-ANIDRO,2012-01-01,AL,ANIDRO,64751.225
4,2012-01-01,AL,HIDRATADO,3766.0,2012-01-01-AL-ETANOL HIDRATADO,2012-01-01,AL,HIDRATADO,0.000
5,2012-01-01,AL,HIDRATADO,0.0,2012-01-01-AL-HIDRATADO,2012-01-01,AL,HIDRATADO,51371.502
6,2012-01-01,AM,ANIDRO,0.0,2012-01-01-AM-ANIDRO,2012-01-01,AM,ANIDRO,0.000
7,2012-01-01,AM,HIDRATADO,2913.0,2012-01-01-AM-ETANOL HIDRATADO,2012-01-01,AM,HIDRATADO,0.000
8,2012-01-01,AM,HIDRATADO,0.0,2012-01-01-AM-HIDRATADO,2012-01-01,AM,HIDRATADO,0.000
9,2012-01-01,AP,ANIDRO,0.0,2012-01-01-AP-ANIDRO,2012-01-01,AP,ANIDRO,0.000


In [66]:
supply_and_demand_SandD_ethanol.tail(20)

,Date_d,UF_d,PRODUTO_d,VENDAS,KEY,Date_s,UF_s,PRODUTO_s,PRODUÇÃO
11776,2024-12-01,RO,HIDRATADO,4016.733,2024-12-01-RO-ETANOL HIDRATADO,2024-12-01,RO,HIDRATADO,0.000
11777,2024-12-01,RO,HIDRATADO,0.000,2024-12-01-RO-HIDRATADO,2024-12-01,RO,HIDRATADO,0.000
11778,2024-12-01,RR,ANIDRO,4582.380,2024-12-01-RR-ANIDRO,2024-12-01,RR,ANIDRO,0.000
11779,2024-12-01,RR,HIDRATADO,1192.000,2024-12-01-RR-ETANOL HIDRATADO,2024-12-01,RR,HIDRATADO,0.000
11780,2024-12-01,RR,HIDRATADO,0.000,2024-12-01-RR-HIDRATADO,2024-12-01,RR,HIDRATADO,0.000
11781,2024-12-01,RS,ANIDRO,99437.760,2024-12-01-RS-ANIDRO,2024-12-01,RS,ANIDRO,0.000
11782,2024-12-01,RS,HIDRATADO,12879.915,2024-12-01-RS-ETANOL HIDRATADO,2024-12-01,RS,HIDRATADO,0.000
11783,2024-12-01,RS,HIDRATADO,0.000,2024-12-01-RS-HIDRATADO,2024-12-01,RS,HIDRATADO,0.430
11784,2024-12-01,SC,ANIDRO,80070.430,2024-12-01-SC-ANIDRO,2024-12-01,SC,ANIDRO,0.000
11785,2024-12-01,SC,HIDRATADO,31126.510,2024-12-01-SC-ETANOL HIDRATADO,2024-12-01,SC,HIDRATADO,0.000


In [67]:
supply_and_demand_SandD_ethanol.to_csv('[S&D_ETHANOL].csv')

In [68]:
demand_SanD.to_csv('[S&D_Demand_EthaGas].csv')

In [69]:
UF_d = demand['UNIDADE DA FEDERAÇÃO'].drop_duplicates()
UF_s = supply['UNIDADE DA FEDERAÇÃO'].drop_duplicates()
ufs_csv = pd.concat([UF_d,UF_d],ignore_index=True).drop_duplicates().sort_values(ascending=True).reset_index()
ufs_csv['UF'] = ufs_csv['UNIDADE DA FEDERAÇÃO'].map(federative_unit)
ufs_csv.to_csv('[S&D_dim_UF].csv')
